# MLPerf Inference v2.0 - Reproducing Orin results with JetPack 5.0.1 DP

# Table of Contents

1. [System](#system)
1. [Installation](#installation)
    1. [Clone the repo](#installation_repo)
    1. [Install the dependencies](#installation_deps)
    1. [Link the datasets](#installation_datasets)
    1. [Download the models](#installation_models)
    1. [Build the harness](#installation_harness)
1. [ResNet50](#resnet50)
    1. [Offline](#resnet50_offline)
        1. [Build](#resnet50_offline_build)
        1. [Performance](#resnet50_offline_performance)
        1. [Accuracy](#resnet50_offline_accuracy)
    1. [SingleStream](#resnet50_singlestream)
        1. [Build](#resnet50_singlestream_build)
        1. [Performance](#resnet50_singlestream_performance)
        1. [Accuracy](#resnet50_singlestream_accuracy)
    1. [MultiStream](#resnet50_multistream)
        1. [Build](#resnet50_multistream_build)
        1. [Performance](#resnet50_multistream_performance)
        1. [Accuracy](#resnet50_multistream_accuracy)
1. [SSD-ResNet34](#ssd-resnet34)
    1. [Offline](#ssd-resnet34_offline)
        1. [Build](#ssd-resnet34_offline_build)
        1. [Performance](#ssd-resnet34_offline_performance)
        1. [Accuracy](#ssd-resnet34_offline_accuracy)
    1. [SingleStream](#ssd-resnet34_singlestream)
        1. [Build](#ssd-resnet34_singlestream_build)
        1. [Performance](#ssd-resnet34_singlestream_performance)
        1. [Accuracy](#ssd-resnet34_singlestream_accuracy)
    1. [MultiStream](#ssd-resnet34_multistream)
        1. [Build](#ssd-resnet34_multistream_build)
        1. [Performance](#ssd-resnet34_multistream_performance)
        1. [Accuracy](#ssd-resnet34_multistream_accuracy)

<a name="system"></a>
# System: [NVIDIA Orin](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/systems/Orin_TRT.json)

## Stats: JetPack

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo -H python3 -m pip install jetson-stats -U
...
Successfully installed jetson-stats-3.1.0
</pre>

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; jetson_release

 - NVIDIA Jetson UNKNOWN
   * Jetpack UNKNOWN [L4T 34.1.0]
   * NV Power Mode: MAXN - Type: 0
   * jetson_stats.service: active
 - Libraries:
   * CUDA: NOT_INSTALLED
   * cuDNN: 8.3.2.49
   * TensorRT: 8.4.0.9
   * Visionworks: NOT_INSTALLED
   * OpenCV: 4.5.4 compiled CUDA: NO
   * VPI: ii libnvvpi2 2.0.9 arm64 NVIDIA Vision Programming Interface library
   * Vulkan: 1.3.203
</pre>

## OS: Ubuntu 18.04.5 LTS

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cat /etc/lsb-release
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION=&quot;Ubuntu 20.04.4 LTS&quot;
</pre>

## CPU: 12-core ARMv81 @ 2200 MHz

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; lscpu
Architecture:                    aarch64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
CPU(s):                          12
On-line CPU(s) list:             0-11
Thread(s) per core:              1
Core(s) per socket:              4
Socket(s):                       3
Vendor ID:                       ARM
Model:                           1
Model name:                      ARMv8 Processor rev 1 (v8l)
Stepping:                        r0p1
CPU max MHz:                     2201.6001
CPU min MHz:                     115.2000
BogoMIPS:                        62.50
L1d cache:                       768 KiB
L1i cache:                       768 KiB
L2 cache:                        3 MiB
L3 cache:                        6 MiB
Vulnerability Itlb multihit:     Not affected
Vulnerability L1tf:              Not affected
Vulnerability Mds:               Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl
Vulnerability Spectre v1:        Mitigation; __user pointer sanitization
Vulnerability Spectre v2:        Not affected
Vulnerability Srbds:             Not affected
Vulnerability Tsx async abort:   Not affected
Flags:                           fp asimd evtstrm aes pmull sha1 sha2 crc32 atomics fphp asimdhp cpuid asimdrdm lrcpc dcpop asimddp uscat ilrcpc flagm
</pre>

## GPU

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; sudo jetson_clocks --show
SOC family:tegra234  Machine:NVIDIA Orin Jetson-Small Developer Kit
Online CPUs: 0-11
cpu0: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu1: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu10: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu11: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu2: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu3: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu4: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu5: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu6: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu7: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu8: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
cpu9: Online=1 Governor=schedutil MinFreq=2201600 MaxFreq=2201600 CurrentFreq=2201600 IdleStates: WFI=0 c7=0
GPU MinFreq=1300500000 MaxFreq=1300500000 CurrentFreq=1300500000
EMC MinFreq=204000000 MaxFreq=3199000000 CurrentFreq=3199000000 FreqOverride=1
DLA0_CORE MinFreq=0 MaxFreq=1536000000 CurrentFreq=1536000000
DLA0_FALCON MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
DLA1_CORE MinFreq=0 MaxFreq=1536000000 CurrentFreq=1536000000
DLA1_FALCON MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
PVA0_VPS0 MinFreq=0 MaxFreq=1152000000 CurrentFreq=1152000000
PVA0_AXI MinFreq=0 MaxFreq=832000000 CurrentFreq=832000000
FAN Dynamic Speed control=active hwmon3_pwm=88
NV Power Mode: MAXN
</pre>


<pre>
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck compile program:tool-print-cuda-devices
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; ck run program:tool-print-cuda-devices
...
GPU Device ID: 0
GPU Name: Orin
GPU compute capability: 8.7
CUDA driver version: 11.4
CUDA runtime version: 11.4
Global memory: 32110174208
Max clock rate: 1300.000000 MHz
Total amount of shared memory per block: 49152
Total number of registers available per block: 65536
Warp size: 32
Maximum number of threads per multiprocessor:  1536
Maximum number of threads per block: 1024
Max dimension size of a thread block X: 1024
Max dimension size of a thread block Y: 1024
Max dimension size of a thread block Z: 64
Max dimension size of a grid size X: 2147483647
Max dimension size of a grid size Y: 65535
Max dimension size of a grid size Z: 65535
</pre>


## Disks

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; df -h
Filesystem      Size  Used Avail Use% Mounted on
/dev/mmcblk0p1   57G   18G   37G  33% /
none             15G     0   15G   0% /dev
tmpfs            15G   52K   15G   1% /dev/shm
tmpfs           3.0G   11M  3.0G   1% /run
tmpfs           5.0M  4.0K  5.0M   1% /run/lock
tmpfs            15G     0   15G   0% /sys/fs/cgroup
/dev/mmcblk1p1  469G   36G  409G   9% /sd
tmpfs           3.0G   16K  3.0G   1% /run/user/124
tmpfs           3.0G  4.0K  3.0G   1% /run/user/1000
tmpfs           3.0G  8.0K  3.0G   1% /run/user/1001
</pre>

<a name="installation"></a>
# Installation

<a name="installation_repo"></a>
## Clone the v2.0 results repo

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; git clone https://github.com/mlcommons/inference_results_v2.0.git</pre>

<a name="installation_deps"></a>
## Install the dependencies

To [quote](https://github.com/mlcommons/inference_results_v2.0/tree/master/closed/Azure) Azure:
> Note that this might take a while, on the order of several hours.

<pre><font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; chmod +x inference_results_v2.0/closed/Azure/scripts/install_orin_jetson_dependencies.sh
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; time sudo inference_results_v2.0/closed/Azure/scripts/install_orin_jetson_dependencies.sh
<font color="#859900"><b>anton@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; python3 -m pip install onnx
</pre>

<a name="installation_datasets"></a>

<pre>
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; echo 'export MLPERF_SCRATCH_PATH=/datasets/nvidia_scratch' >> $HOME/.bashrc
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; source $HOME/.bashrc
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; mkdir &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>~</b></font>&dollar; cd &dollar;MLPERF_SCRATCH_PATH
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/nvidia_scratch</b></font>&dollar; mkdir data models preprocessed_data
<font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/mlperf_scratch_path</b></font>&dollar; ls -la /datasets/nvidia_scratch/
total 20
drwxrwsr-x  3 anton dvdt 4096 Jan 28 11:10 <font color="#268BD2"><b>.</b></font>
drwxrwsr-x 17 root  dvdt 4096 Jan 28 09:53 <font color="#268BD2"><b>..</b></font>
lrwxrwxrwx  1 anton dvdt   64 Jan 28 11:10 <font color="#2AA198"><b>data</b></font> 
drwxrwsr-x  5 anton dvdt 4096 Jan 28 09:54 <font color="#268BD2"><b>models</b></font>
lrwxrwxrwx  1 anton dvdt   77 Jan 28 11:10 <font color="#2AA198"><b>preprocessed_data</b></font> 
</pre>

<a name="download_datasets"></a>
## Download the datasets
<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make download_data BENCHMARKS="bert ssd-mobilenet"

<a name="installation_models"></a>
## Download the models

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make download_model BENCHMARKS="resnet50 bert ssd-mobilenet ssd-resnet34"
...
Finished downloading all the models!
</pre>

## Preprocess the dataset

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>:/datasets/inference_results_v2.0/closed/Azure</b></font>&dollar; make preprocess_data BENCHMARKS="resnet50 bert ssd-mobilenet ssd-resnet34"
</pre>

## Build the binaries

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; git diff code/harness/CMakeLists.txt

diff --git a/closed/NVIDIA/code/harness/CMakeLists.txt b/closed/NVIDIA/code/harness/CMakeLists.txt
index 074c0bc25..b205e7691 100644
--- a/closed/NVIDIA/code/harness/CMakeLists.txt
+++ b/closed/NVIDIA/code/harness/CMakeLists.txt
@@ -503,48 +503,48 @@ else()
     )

     ######### RNN-T HARNESS ########
-    execute_process(COMMAND echo "Building RNN-T harness...")
+    #    execute_process(COMMAND echo "Building RNN-T harness...")

-    get_dali_paths(DALI_INCLUDE_DIR DALI_LIB_DIR DALI_LIBRARIES)
+    #    get_dali_paths(DALI_INCLUDE_DIR DALI_LIB_DIR DALI_LIBRARIES)

-    message(STATUS "DALI libraries DIR: " ${DALI_LIB_DIR})
-    message(STATUS "DALI include DIR: " ${DALI_INCLUDE_DIR})
+    #    message(STATUS "DALI libraries DIR: " ${DALI_LIB_DIR})
+    #    message(STATUS "DALI include DIR: " ${DALI_INCLUDE_DIR})

-    message(STATUS "DALI linked libraries: " ${DALI_LIBRARIES})
+    #    message(STATUS "DALI linked libraries: " ${DALI_LIBRARIES})

-    add_executable(harness_rnnt
-        harness_rnnt/main_rnnt.cc
-        harness_rnnt/rnnt_kernels.cu
-        common/logger.cpp
-    )
+    #    add_executable(harness_rnnt
+    #        harness_rnnt/main_rnnt.cc
+    #        harness_rnnt/rnnt_kernels.cu
+    #        common/logger.cpp
+    #    )

     # Add gencode compilation option for SOC (Xavier, Orin)
-    if (${IS_SOC})
-        message(STATUS "Compiling RNNT harness for SM${SOC_SM}.")
-        target_compile_options(harness_rnnt PRIVATE $<$<COMPILE_LANGUAGE:CUDA>: -gencode arch=compute_${SOC_SM},code=sm_${SOC_SM}>)
-    endif()
-
-    target_link_directories(harness_rnnt PRIVATE ${DALI_LIB_DIR})
-
-    target_link_libraries(harness_rnnt
-        nvinfer
-        nvinfer_plugin
-        gflags
-        glog
-        ${CUDA_LIBRARIES}
-        ${LOADGEN_LIB}
-        ${DALI_LIBRARIES}
-    )
-
-    target_include_directories(harness_rnnt
-        PUBLIC
-            ${CUDA_INCLUDE_DIRS}
-            ${LOADGEN_INCLUDE_DIR}
-            ${LWIS_INCLUDE_DIR}
-            ${DALI_INCLUDE_DIR}
-            common
-            harness_rnnt
-    )
+    #    if (${IS_SOC})
+    #        message(STATUS "Compiling RNNT harness for SM${SOC_SM}.")
+    #        target_compile_options(harness_rnnt PRIVATE $<$<COMPILE_LANGUAGE:CUDA>: -gencode arch=compute_${SOC_SM},code=sm_${SOC_SM}>)
+    #    endif()
+
+    #    target_link_directories(harness_rnnt PRIVATE ${DALI_LIB_DIR})
+
+    #    target_link_libraries(harness_rnnt
+    #        nvinfer
+    #        nvinfer_plugin
+    #        gflags
+    #        glog
+    #        ${CUDA_LIBRARIES}
+    #        ${LOADGEN_LIB}
+    #        ${DALI_LIBRARIES}
+    #    )
+
+    #    target_include_directories(harness_rnnt
+    #        PUBLIC
+    #            ${CUDA_INCLUDE_DIRS}
+    #            ${LOADGEN_INCLUDE_DIR}
+    #            ${LWIS_INCLUDE_DIR}
+    #            ${DALI_INCLUDE_DIR}
+    #            common
+    #            harness_rnnt
+    #    )


<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make build
...
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>$ ls -la build/
total 28

total 40
drwxrwsr-x 10 katya krai 4096 May 19 16:36 .
drwxrwsr-x 15 katya krai 4096 May 19 15:35 ..
drwxr-sr-x  2 katya krai 4096 May 19 16:11 bin
lrwxrwxrwx  1 katya krai   29 May 19 16:36 data -> /datasets/nvidia_scratch/data
drwxr-sr-x  3 katya krai 4096 May 19 14:20 engines
drwxrwsr-x  5 katya krai 4096 May 19 16:11 harness
drwxrwsr-x 14 katya krai 4096 May 19 11:22 inference
drwxrwsr-x  8 katya krai 4096 May 19 16:36 logs
lrwxrwxrwx  1 katya krai   31 May 19 16:36 models -> /datasets/nvidia_scratch/models
drwxrwsr-x  6 katya krai 4096 May 19 11:36 plugins
drwxrwsr-x  9 katya krai 4096 May 19 11:22 power-dev
lrwxrwxrwx  1 katya krai   42 May 19 16:36 preprocessed_data -> /datasets/nvidia_scratch/preprocessed_data
drwxrwsr-x 10 katya krai 4096 May 19 11:23 triton-inference-server

</pre>

<a name="resnet50"></a>
# ResNet50

<a name="resnet50_offline"></a>
## Offline

<a name="resnet50_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=offline&quot;
...
[2022-05-20 10:41:20,200 main.py:137 INFO] Finished building engines for resnet50 benchmark in Offline scenario.
Time taken to generate engines: 265.04155135154724 seconds

</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-Offline*.plan -exec du -hs {} \;
27M     ./build/engines/Orin/resnet50/Offline/resnet50-Offline-gpu-b256-int8.lwis_k_99_MaxP.plan
26M     ./build/engines/Orin/resnet50/Offline/resnet50-Offline-dla-b8-int8.lwis_k_99_MaxP.plan
</pre>

<a name="resnet50_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 6138.84
Result is : VALID
</pre>

#### Reproduced experiment (after system reboot!)

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=Offline --test_mode=PerformanceOnly&quot;
...

[2022-05-22 23:59:24,613 main.py:304 INFO] Result: result_samples_per_second: 5738.38, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: result_samples_per_second: 5738.38, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: No accuracy results in PerformanceOnly mode.


real    11m7.798s
user    11m1.016s
sys     0m9.948s
</pre>

<a name="resnet50_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/Offline/accuracy/accuracy.txt)

<pre>
accuracy=76.038%, good=38019, total=50000
hash=070aa1cee196b65944c4d0f7778135ab66c849bf1374eb4ccbc0aa43cddbc9ff
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
accuracy=76.038%, good=38019, total=50000 TO UPDATE

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    resnet50: Accuracy = 76.038, Threshold = 75.695. Accuracy test PASSED.
</pre>

<a name="resnet50_singlestream"></a>
## SingleStream

<a name="resnet50_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=singlestream&quot;
...
[2022-05-24 13:06:32,113 main.py:137 INFO] Finished building engines for resnet50 benchmark in SingleStream scenario.
Time taken to generate engines: 40.654820680618286 seconds

real    0m44.579s
user    0m21.267s
sys     0m7.008s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-SingleStream*.plan -exec du -hs {} \;
27M     ./build/engines/Orin/resnet50/SingleStream/resnet50-SingleStream-gpu-b1-int8.lwis_k_99_MaxP.plan
</pre>

<a name="resnet50_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 694827
Result is : VALID
  Min duration satisfied : Yes
  Min queries satisfied : Yes
  Early stopping satisfied: Yes
Early Stopping Result:
 * Processed at least 64 queries (861139).
 * Would discard 85465 highest latency queries.
 * Early stopping 90th percentile estimate: 694858
 * Early stopping 99th percentile estimate: 698315
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    resnet50: result_90.00_percentile_latency_ns: 724114, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    resnet50: No accuracy results in PerformanceOnly mode.


real    10m19.950s
user    1m41.248s
sys     0m40.173s

</pre>

<a name="resnet50_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/SingleStream/accuracy/accuracy.txt)

<pre>
accuracy=76.030%, good=38015, total=50000
hash=f7aabe1e91e185d900dc060ca6e7f0fc2f57b1b5cbd447c31411cf06a93150b4
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.040%, good=38020, total=50000

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    resnet50: Accuracy = 76.040, Threshold = 75.695. Accuracy test PASSED.


real    3m49.734s
user    0m31.405s
sys     0m16.638s
</pre>

<a name="resnet50_multistream"></a>
## MultiStream

<a name="resnet50_multistream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=multistream&quot;
...
[2022-05-24 13:08:06,236 main.py:137 INFO] Finished building engines for resnet50 benchmark in MultiStream scenario.
Time taken to generate engines: 34.90856742858887 seconds

real    0m38.929s
user    0m19.816s
sys     0m8.275s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name resnet50-MultiStream*.plan -exec du -hs {} \;
27M     ./build/engines/Orin/resnet50/MultiStream/resnet50-MultiStream-gpu-b8-int8.lwis_k_99_MaxP.plan
</pre>

<a name="resnet50_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
99th percentile latency (ns) : 3956535
Result is : VALID
</pre>

#### Our experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;


======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    resnet50: result_99.00_percentile_per_query_latency_ns: 2352684, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    resnet50: No accuracy results in PerformanceOnly mode.


real    10m17.854s
user    1m23.250s
sys     0m17.760s
</pre>

<a name="resnet50_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/resnet50/MultiStream/accuracy/accuracy.txt)

<pre>
accuracy=76.030%, good=38015, total=50000
hash=acf6aabdb21f556665ebb5dcbf04a11228fea64368c4485ce325f2362e00d909
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=resnet50 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
accuracy=76.040%, good=38020, total=50000

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    resnet50: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    resnet50: Accuracy = 76.040, Threshold = 75.695. Accuracy test PASSED.


real    2m19.813s
user    0m25.461s
sys     0m11.814s
</pre>

<a name="bert"></a>
# BERT

<a name="bert_offline"></a>
## Offline

<a name="bert_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=bert --scenarios=offline&quot;
...
Time taken to generate engines: 4379.266929149628 seconds

real    73m3.881s
user    0m32.668s
sys     16m22.002s
</pre>

<pre><font color="#859900"><b>anton@xavier</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name bert-Offline*.plan -exec du -hs {} \;
355M    ./build/engines/Orin/bert/Offline/bert-Offline-gpu-int8_S_384_B_256_P_1_vs_il.custom_k_99_MaxP.plan
</pre>

<a name="bert_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 476.344
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=bert --scenarios=Offline --test_mode=PerformanceOnly&quot;
...

[2022-05-23 02:00:20,632 main.py:304 INFO] Result: result_samples_per_second: 482.61, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: result_samples_per_second: 482.61, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: No accuracy results in PerformanceOnly mode.


real    11m18.494s
user    0m9.500s
sys     0m2.521s
</pre>

<a name="bert_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/Offline/accuracy/accuracy.txt)

<pre>
{"exact_match": 82.687, "f1": 90.104}
hash=f56718adf61873042f22ce72c152f015b8ac1057a658676ba39c505c2b6f22cf
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=bert --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-Offline:
    bert: Accuracy = 90.104, Threshold = 89.965. Accuracy test PASSED.


real    1m54.490s
user    1m17.205s
sys     0m1.996s
</pre>

<a name="bert_singlestream"></a>
## SingleStream

<a name="bert_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=bert --scenarios=singlestream&quot;
...
[2022-05-24 14:43:33,052 main.py:137 INFO] Finished building engines for bert benchmark in SingleStream scenario.
Time taken to generate engines: 67.85338068008423 seconds

real    1m12.345s
user    0m33.929s
sys     0m12.486s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name bert-SingleStream*.plan -exec du -hs {} \;
354M    ./build/engines/Orin/bert/SingleStream/bert-SingleStream-gpu-int8_S_384_B_1_P_1_vs.custom_k_99_MaxP.plan
</pre>

<a name="bert_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 7634138
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=bert --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...

[2022-05-24 14:58:48,681 main.py:304 INFO] Result: result_90.00_percentile_latency_ns: 8173938, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-custom_k_99_MaxP-SingleStream:
    bert: result_90.00_percentile_latency_ns: 8173938, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-SingleStream:
    bert: No accuracy results in PerformanceOnly mode.


real    10m6.405s
user    4m2.775s
sys     0m6.617s
</pre>

<a name="bert_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/bert-99/SingleStream/accuracy/accuracy.txt)

<pre>
{"exact_match": 83.245, "f1": 90.445}
hash=76ded78de16cdd33c6a8f95ec5a92907234ae8bf850057bc36dc90d0f22fbbd6
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=bert --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...

======================= Accuracy results: =======================

Orin_TRT-custom_k_99_MaxP-SingleStream:
    bert: Accuracy = 90.440, Threshold = 89.965. Accuracy test PASSED.


real    2m29.181s
user    1m46.591s
sys     0m2.966s
</pre>

<a name="ssd-mobilenet"></a>
# SSD-MobileNet

<a name="ssd-mobilenet_offline"></a>
## Offline

<a name="ssd-mobilenet_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=offline&quot;
...
[2022-05-24 16:16:00,182 main.py:137 INFO] Finished building engines for ssd-mobilenet benchmark in Offline scenario.
Time taken to generate engines: 352.52208948135376 seconds

real    5m57.185s
user    3m14.667s
sys     1m15.765s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-mobilenet-Offline*.plan -exec du -hs {} \;
23M     ./build/engines/Orin/ssd-mobilenet/Offline/ssd-mobilenet-Offline-dla-b64-int8.lwis_k_99_MaxP.plan
8.3M    ./build/engines/Orin/ssd-mobilenet/Offline/ssd-mobilenet-Offline-gpu-b128-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-mobilenet_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 6883.49
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=Offline --test_mode=PerformanceOnly&quot;
...

[2022-05-24 16:31:29,790 main.py:304 INFO] Result: result_samples_per_second: 6744.67, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    ssd-mobilenet: result_samples_per_second: 6744.67, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    ssd-mobilenet: No accuracy results in PerformanceOnly mode.


real    11m53.101s
user    11m40.497s
sys     0m4.586s
</pre>

<a name="ssd-mobilenet_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/Offline/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.523
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.207
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.188
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.595
mAP=22.861%
hash=1b8328272ef4ff9d38b6640151584af6d9e7db27cc7ca74830fd8e95ad4835e5
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    ssd-mobilenet: Accuracy = 22.876, Threshold = 21.780. Accuracy test PASSED.


real    1m3.479s
user    0m27.640s
sys     0m3.919s
</pre>

<a name="ssd-mobilenet_singlestream"></a>
## SingleStream

<a name="ssd-mobilenet_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=singlestream&quot;
...
[2022-05-24 16:56:19,282 main.py:137 INFO] Finished building engines for ssd-mobilenet benchmark in SingleStream scenario.
Time taken to generate engines: 75.95014977455139 seconds

real    1m20.457s
user    1m1.072s
sys     0m12.098s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-mobilenet-SingleStream*.plan -exec du -hs {} \;
8.4M    ./build/engines/Orin/ssd-mobilenet/SingleStream/ssd-mobilenet-SingleStream-gpu-b1-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-mobilenet_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 473641
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...

[2022-05-24 17:13:37,435 main.py:304 INFO] Result: result_90.00_percentile_latency_ns: 531497, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-mobilenet: result_90.00_percentile_latency_ns: 531497, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-mobilenet: No accuracy results in PerformanceOnly mode.


real    10m13.537s
user    2m58.244s
sys     0m52.462s
</pre>

<a name="ssd-mobilenet_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/SingleStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.253
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.525
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.207
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.021
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.189
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.598
mAP=22.914%
hash=52c4ac8a051c244379e3f497330550e84b976557577827f5e290c4873b56594a
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...

======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-mobilenet: Accuracy = 22.914, Threshold = 21.780. Accuracy test PASSED.


real    0m37.466s
user    0m27.414s
sys     0m6.092s
</pre>

<a name="ssd-mobilenet_multistream"></a>
## MultiStream

<a name="ssd-mobilenet_multistream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=multistream&quot;
...
[2022-05-24 17:01:32,444 main.py:137 INFO] Finished building engines for ssd-mobilenet benchmark in MultiStream scenario.
Time taken to generate engines: 81.45282483100891 seconds

real    1m25.974s
user    0m59.426s
sys     0m14.775s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-mobilenet-MultiStream*.plan -exec du -hs {} \;
8.1M    ./build/engines/Orin/ssd-mobilenet/MultiStream/ssd-mobilenet-MultiStream-gpu-b8-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-mobilenet_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
99th percentile latency (ns) : 4266439
Result is : VALID
</pre>

#### Our experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
...

[2022-05-24 17:45:44,125 main.py:304 INFO] Result: result_99.00_percentile_per_query_latency_ns: 2041312, Result is VALID

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-mobilenet: result_99.00_percentile_per_query_latency_ns: 2041312, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-mobilenet: No accuracy results in PerformanceOnly mode.


real    10m11.801s
user    2m42.124s
sys     0m15.989s
</pre>

<a name="ssd-mobilenet_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-mobilenet/MultiStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.253
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.525
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.207
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.021
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.189
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.598
mAP=22.914%
hash=53cc6074ee0c9a5510f9d190cc9b73e6b52e3ffc2d3aba6db4e93d4a6c7934e1
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-mobilenet --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...

======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-mobilenet: Accuracy = 22.914, Threshold = 21.780. Accuracy test PASSED.


real    0m36.379s
user    0m27.289s
sys     0m5.944s
</pre>

<a name="ssd-resnet34"></a>
# SSD-ResNet34

<a name="ssd-resnet34_offline"></a>
## Offline

<a name="ssd-resnet34_offline_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=offline&quot;
...
[2022-05-24 09:37:25,912 main.py:137 INFO] Finished building engines for ssd-resnet34 benchmark in Offline scenario.
Time taken to generate engines: 577.0244572162628 seconds
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-Offline*.plan -exec du -hs {} \;
30M     ./build/engines/Orin/ssd-resnet34/Offline/ssd-resnet34-Offline-dla-b8-int8.lwis_k_99_MaxP.plan
23M     ./build/engines/Orin/ssd-resnet34/Offline/ssd-resnet34-Offline-gpu-b8-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-resnet34_offline_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/Offline/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
Samples per second: 207.657
Result is : VALID
</pre>

#### Our experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=PerformanceOnly&quot;

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    ssd-resnet34: result_samples_per_second: 177.848, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-Offline:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    11m40.392s
user    11m31.448s
sys     0m3.255s
</pre>

<a name="ssd-resnet34_offline_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/Offline/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.254
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.330
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
mAP=19.909%
hash=615698dac9c5f0889acd9c626a5241781466b9699c8e57ff1773ce2739f310bf
</pre>

#### Our experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=Offline --test_mode=AccuracyOnly&quot;
...
  Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.185
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.236
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.202
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.329
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.178
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.426
mAP=19.746%
Traceback (most recent call last):
  File "code/main.py", line 772, in <module>
    main(main_args, DETECTED_SYSTEM)
  File "code/main.py", line 744, in main
    dispatch_action(main_args, config_dict, workload_id, equiv_engine_setting=equiv_engine_setting)
  File "code/main.py", line 574, in dispatch_action
    handle_run_harness(benchmark_conf, need_gpu, need_dla, profile, power)
  File "code/main.py", line 339, in handle_run_harness
    accuracy = check_accuracy(os.path.join(harness.get_full_log_dir(), "mlperf_log_accuracy.json"), config)
  File "code/main.py", line 497, in check_accuracy
    raise RuntimeError(
RuntimeError: Accuracy = 19.746, Threshold = 19.800. Accuracy test FAILED!
make: *** [Makefile:699: run_harness] Error 1

<a name="ssd-resnet34_singlestream"></a>
## SingleStream

<a name="ssd-resnet34_singlestream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=singlestream&quot;
...
[2022-05-24 10:28:50,980 main.py:137 INFO] Finished building engines for ssd-resnet34 benchmark in SingleStream scenario.
Time taken to generate engines: 164.7405767440796 seconds

real    2m48.949s
user    2m29.160s
sys     1m30.652s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-SingleStream*.plan -exec du -hs {} \;
26M     ./build/engines/Orin/ssd-resnet34/SingleStream/ssd-resnet34-SingleStream-gpu-b1-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-resnet34_singlestream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/SingleStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
90th percentile latency (ns) : 8028243
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=PerformanceOnly&quot;
...
======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-resnet34: result_90.00_percentile_latency_ns: 8130504, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    10m10.330s
user    0m43.939s
sys     0m8.899s
</pre>

<a name="ssd-resnet34_singlestream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/SingleStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.239
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.113%
hash=267e70f78d746992c5142008643dd2f2d9d95424715491aabfe63b3e68e4893c
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=SingleStream --test_mode=AccuracyOnly&quot;
...
  Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.413
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
mAP=19.913%

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-SingleStream:
    ssd-resnet34: Accuracy = 19.913, Threshold = 19.800. Accuracy test PASSED.


real    7m11.931s
user    2m28.669s
sys     0m24.538s
</pre>

<a name="ssd-resnet34_multistream"></a>
## MultiStream

<a name="ssd-resnet34_multistream_build"></a>
### Build

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make generate_engines RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=multistream&quot;
...
[2022-05-24 10:47:04,101 main.py:137 INFO] Finished building engines for ssd-resnet34 benchmark in MultiStream scenario.
Time taken to generate engines: 185.63524317741394 seconds

real    3m9.988s
user    2m29.871s
sys     1m40.165s
</pre>

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; find . \
-name ssd-resnet34-MultiStream*.plan -exec du -hs {} \;
25M     ./build/engines/Orin/ssd-resnet34/MultiStream/ssd-resnet34-MultiStream-gpu-b2-int8.lwis_k_99_MaxP.plan
</pre>

<a name="ssd-resnet34_multistream_performance"></a>
### Performance

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/MultiStream/performance/run_1/mlperf_log_summary.txt#L7)

<pre>
99th percentile latency (ns) : 60667403
Result is : VALID
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin/b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=PerformanceOnly&quot;
...


======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-resnet34: result_99.00_percentile_per_query_latency_ns: 61004829, Result is VALID


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-resnet34: No accuracy results in PerformanceOnly mode.


real    60m10.226s
user    45m14.797s
sys     0m9.696s
</pre>

<a name="ssd-resnet34_multistream_accuracy"></a>
### Accuracy

#### [Submitted experiment](https://github.com/mlcommons/inference_results_v2.0/blob/master/closed/NVIDIA/results/Orin_TRT/ssd-resnet34/MultiStream/accuracy/accuracy.txt)

<pre>
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.239
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.332
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.430
mAP=20.113%
hash=419a56265f86193ebaf1e53c39c8b5bf308f9f99651f4d1b615b0468179bd44c
</pre>

#### Reproduced experiment

<pre><font color="#859900"><b>katya@orin</b></font>:<font color="#268BD2"><b>/datasets/inference_results_v2.0/closed/NVIDIA</b></font>&dollar; time \
make run_harness RUN_ARGS=&quot;--benchmarks=ssd-resnet34 --scenarios=MultiStream --test_mode=AccuracyOnly&quot;
...
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.253
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.239
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.178
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.429
mAP=19.891%

======================= Perf harness results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-resnet34: Cannot find performance result. Maybe you are running in AccuracyOnly mode.


======================= Accuracy results: =======================

Orin_TRT-lwis_k_99_MaxP-MultiStream:
    ssd-resnet34: Accuracy = 19.891, Threshold = 19.800. Accuracy test PASSED.


real    4m30.403s
user    2m51.865s
sys     0m17.264s
</pre>